In [1]:
# """深度学习建模流程

# stage 1 模型选择    : 确定神经网络的层数，每层神经元的个数以及激活函数
# stage 2 确定目标函数 : 构建包含模型参数的函数方程，函数取值与建模目标一致，大多情况求解方程的极小值
# stage 3 选择优化算法 : 根据损失函数的函数特性，选择最优化算法，以减少算力消耗
# stage 4 模型训练    : 利用优化算法求解损失函数得到模型参数（连接神经元的参数取值）

# """

# 线性回归建模

## 手动实现

In [3]:
# 随机模块
import random

# 绘图模块
import matplotlib as mpl
import matplotlib.pyplot as plt

# numpy
import numpy as np

# pytorch
import torch
from torch import nn,optim
import torch.nn.functional as F
from torch.utils.data import Dataset,TensorDataset,DataLoader
from torch.utils.tensorboard import SummaryWriter

# 导入自定义模块
from torchLearning import *

In [4]:
tensorGenReg?

Signature: tensorGenReg(num_examples=1000, w=[2, -1, 1], bias=True, delta=0.01, deg=1)
Docstring:
回归类数据集创建函数

param num_examples:创建数据集的数据量
param w:包括截距的特征系数向量
param bias:是否需要截距
param delta:扰动项取值
param def:方程次数
return :生成的特征张量和标签张量

该函数无法创建带有交叉项的方程 eg:deg>=2 y=x1^2 + x1x2(交叉项) + x2^2
File:      ~/Projects/pytorch_learning/torchLearning.py
Type:      function


In [5]:
# 生成数据集 y = 2x1 - x2 +1
torch.manual_seed(420)

features,labels = tensorGenReg()

In [6]:
# 建模流程

In [7]:
def linreg(X,w):
    return torch.mm(X,w)

In [44]:
# 确定目标函数
def squared_loss(y_hat,y):
    num_ = y.numel()
    sse = torch.sum((y_hat.reshape(-1,1) - y.reshape(-1,1)) ** 2)
    
    return sse/num_

In [48]:
# 定义优化算法
def sgd(params,lr):
    params.data -= lr*params.grad
    params.grad.zero_()

In [ ]:
# -= 之类的操作：in-place operation(对 原对象 修改操作)

### 关于可微张量的 in-place operation 的相关讨论
(1) 正常情况下，可微张量的 in-place operation 会导致系统无法区分叶节点 和 其他节点的问题

修改后 叶节点 可能不再是 叶节点，计算图是否还会有意义

In [10]:
w = torch.tensor(2.,requires_grad=True)
w

tensor(2., requires_grad=True)

In [15]:
w.is_leaf # w是创建的可微张量，是个叶节点

True

In [ ]:
# 开启可微之后，w的所有计算都会被纳入计算图中

In [13]:
w1 = w * 2
w1

tensor(4., grad_fn=<MulBackward0>)

In [16]:
# 如果在计算过程中，使用in-place operation，让新生成的值替换w原始值，则会报错

In [17]:
w = torch.tensor(2.,requires_grad = True)
w -= w*2

RuntimeError: a leaf Variable that requires grad is being used in an in-place operation.

In [22]:
# 可以使用其他方法对w进行修改，不过w将不再是叶节点，也不能通过反向传播求其导数了

In [23]:
w = torch.tensor(2.,requires_grad = True)
w = w*2
w.is_leaf

False

In [24]:
w.backward() # w已经成为输出节点，即使w存在在当前操作空间中，但没有任何变量指向它，相当于丢失，之后也没办法使用任何方法对其梯度进行查看，计算图也没了存在意义（核心价值）

In [25]:
w.grad

<ipython-input-25-d01581d16022>:1: UserWarning: The .grad attribute of a Tensor that is not a leaf Tensor is being accessed. Its .grad attribute won't be populated during autograd.backward(). If you indeed want the gradient for a non-leaf Tensor, use .retain_grad() on the non-leaf Tensor. If you access the non-leaf Tensor by mistake, make sure you access the leaf Tensor instead. See github.com/pytorch/pytorch/pull/30531 for more informations.
  w.grad


（2）叶节点数值修改方法

   如果出现了一定要修改叶节点取值的情况，典型的如梯度下降过程中利用梯度值修改参数值时，可以使用此前介绍的暂停追踪的方法，如使用with torch.no_grad()语句或者torch.detach_()方法，使得修改叶节点数值时暂停追踪，然后再生成新的叶节点带入计算。

In [31]:
# 利用 with torch.no_grad()暂停追踪

w = torch.tensor(2.,requires_grad = True)

with torch.no_grad():
    w -= w * 2

w,w.is_leaf

(tensor(-2., requires_grad=True), True)

In [34]:
w = torch.tensor(2.,requires_grad = True)

w.detach_()
w -= w * 2

w,w.is_leaf # 注意requires_grad

(tensor(-2.), True)

In [36]:
w.requires_grad = True
w

tensor(-2., requires_grad=True)

In [37]:
# 利用 torch.detach_()暂停追踪

w = torch.tensor(2.,requires_grad = True)

w.detach_()
w -= w * 2
w.requires_grad = True

w,w.is_leaf # 注意requires_grad可微属性

(tensor(-2., requires_grad=True), True)

In [38]:
# 利用.data来返回可微张量的取值，避免在修改的过程中被追踪
w = torch.tensor(2.,requires_grad = True)

w.data # .data查看张量的数值，但不改变张量本身的可微性
print(w.data,w)

w.data -= w * 2 # 对其数值进行修改

w,w.is_leaf

tensor(2.) tensor(2., requires_grad=True)


(tensor(-2., requires_grad=True), True)

In [39]:
# 训练模型

In [49]:
torch.manual_seed(420)

# 初始化核心参数
batch_size = 10 # 每一个小批量数
lr = 0.03 # 学习率
num_epochs = 3 # 训练遍历次数
w = torch.zeros(3,1,requires_grad = True) #随机设置初始权重

# 参与训练的模型方程
net = linreg #使用回归方程 torch.mm(X,w)
loss = squared_loss # MSE作为损失函数

# 模型训练过程
for epoch in range(num_epochs):
    for X,y in data_iter(batch_size,features,labels):
        l = loss(net(X,w),y)
        l.backward()
        sgd(w,lr)
    train_l = loss(net(features,w),labels)
    print('epoch %d,loss %f' % (epoch + 1,train_l))

epoch 1,loss 0.000128
epoch 2,loss 0.000101
epoch 3,loss 0.000103


In [50]:
# 随机模块
import random

# 绘图模块
import matplotlib as mpl
import matplotlib.pyplot as plt

# numpy
import numpy as np

# pytorch
import torch
from torch import nn,optim
import torch.nn.functional as F
from torch.utils.data import Dataset,TensorDataset,DataLoader
from torch.utils.tensorboard import SummaryWriter

# 导入自定义模块
from torchLearning import *

# 生成数据集 y = 2x1 - x2 +1
torch.manual_seed(420)

features,labels = tensorGenReg()

# 建模流程
def linreg(X,w):
    return torch.mm(X,w)

# 确定目标函数
def squared_loss(y_hat,y):
    num_ = y.numel()
    sse = torch.sum((y_hat.reshape(-1,1) - y.reshape(-1,1)) ** 2)
    
    return sse/num_

# 定义优化算法
def sgd(params,lr):
    params.data -= lr*params.grad
    params.grad.zero_()
    
# 训练模型
torch.manual_seed(420)

# 初始化核心参数
batch_size = 10 # 每一个小批量数
lr = 0.03 # 学习率
num_epochs = 3 # 训练遍历次数
w = torch.zeros(3,1,requires_grad = True) #随机设置初始权重

# 参与训练的模型方程
net = linreg #使用回归方程 torch.mm(X,w)
loss = squared_loss # MSE作为损失函数

# 模型训练过程
for epoch in range(num_epochs):
    for X,y in data_iter(batch_size,features,labels):
        l = loss(net(X,w),y)
        l.backward()
        sgd(w,lr)
    train_l = loss(net(features,w),labels)
    print('epoch %d,loss %f' % (epoch + 1,train_l))

epoch 1,loss 0.000129
epoch 2,loss 0.000102
epoch 3,loss 0.000102


In [51]:
w

tensor([[ 2.0003],
        [-1.0002],
        [ 1.0008]], requires_grad=True)

In [58]:
# 随机模块
import random

# 绘图模块
import matplotlib as mpl
import matplotlib.pyplot as plt

# numpy
import numpy as np

# pytorch
import torch
from torch import nn,optim
import torch.nn.functional as F
from torch.utils.data import Dataset,TensorDataset,DataLoader
from torch.utils.tensorboard import SummaryWriter

# 导入自定义模块
from torchLearning import *

# 生成数据集 y = 2x1 - x2 +1
torch.manual_seed(420)

features,labels = tensorGenReg()

# 建模流程
def linreg(X,w):
    return torch.mm(X,w)

# 确定目标函数
def squared_loss(y_hat,y):
    num_ = y.numel()
    sse = torch.sum((y_hat.reshape(-1,1) - y.reshape(-1,1)) ** 2)
    
    return sse/num_

# 定义优化算法
def sgd(params,lr):
    params.data -= lr*params.grad
    params.grad.zero_()
    
# 训练模型
torch.manual_seed(420)
writer = SummaryWriter(log_dir='reg_loss')

# 初始化核心参数
batch_size = 10 # 每一个小批量数
lr = 0.03 # 学习率
num_epochs = 3 # 训练遍历次数
w = torch.zeros(3,1,requires_grad = True) #随机设置初始权重

# 参与训练的模型方程
net = linreg #使用回归方程 torch.mm(X,w)
loss = squared_loss # MSE作为损失函数

# 模型训练过程
for epoch in range(num_epochs):
    for X,y in data_iter(batch_size,features,labels):
        l = loss(net(X,w),y)
        l.backward()
        sgd(w,lr)
    train_l = loss(net(features,w),labels)
    writer.add_scalar('mul',train_l,epoch)

In [ ]:
# terminal启动服务读取文件
# $ tensorboard --logdir="reg_loss"
# localhost:6006

## 快速实现

In [61]:
# 定义核心参数
batch_size = 10 # 每一个小批量数
lr = 0.03 # 学习率
num_epochs = 3 # 训练遍历次数

In [62]:
# 数据准备
torch.manual_seed(420)

# 创建数据集
features,labels = tensorGenReg()
features = features[:, :-1] # 剔除最后全是1的列
data = TensorDataset(features,labels) # 数据封装
batchData = DataLoader(data,batch_size = batch_size,shuffle = True) # 数据加载

In [63]:
features 

tensor([[-0.0070,  0.5044],
        [ 0.6704, -0.3829],
        [ 0.0302,  0.3826],
        ...,
        [-0.9164, -0.6087],
        [ 0.7815,  1.2865],
        [ 1.4819,  1.1390]])

In [68]:
# 定义模型

class LR(nn.Module):
    def __init__(self,in_features=2,out_features=1): # 定义模型的点线结构
        super(LR,self).__init__()
        self.linear = nn.Linear(in_features,out_features)
        
    def forward(self,x):
        out = self.linear(x)
        
        return out

# 实例化模型
LR_model = LR()

In [69]:
# 定义损失函数

criterion = nn.MSELoss()

In [70]:
# 定义优化方法

optimizer = optim.SGD(LR_model.parameters(), lr=0.03)

In [73]:
# 模型训练

def fit(net,criterion,optimizer,batchdata,epochs):
    for epoch in range(epochs):
        for X,y in batchdata:
            yhat = net.forward(X)
            loss = criterion(yhat,y)
            
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
        writer.add_scalar('loss',loss,global_step=epoch)

In [81]:
# 执行模型训练

writer = SummaryWriter(log_dir='reg_loss')

# 设置随机数种子
torch.manual_seed(420)

fit(net=LR_model,
   criterion = criterion,
   optimizer = optimizer,
   batchdata = batchData,
   epochs = num_epochs)

In [82]:
# 查看模型训练结果

LR_model

LR(
  (linear): Linear(in_features=2, out_features=1, bias=True)
)

In [83]:
# 查看模型参数

list(LR_model.parameters())

[Parameter containing:
 tensor([[ 1.9992, -1.0003]], requires_grad=True),
 Parameter containing:
 tensor([0.9994], requires_grad=True)]

In [84]:
# 计算MSE
criterion(LR_model(features),labels)

tensor(0.0001, grad_fn=<MseLossBackward>)

In [85]:
# 可以使用 add_graph 方法，在writer中添加 模型的记录图

writer.add_graph(LR_model, (features,))
# 在graph一栏可以看到拓扑图——模型结构图

### 简单线性回归的局限性

 当函数方程次数增加为2及以上的多项式函数关系 或 数据 扰动项增加时，简单线性回归误差将迅速增大

In [88]:
# deg调大


# 定义核心参数
batch_size = 10 # 每一个小批量数
lr = 0.03 # 学习率
num_epochs = 3 # 训练遍历次数

# 数据准备
torch.manual_seed(420)

# 创建数据集
features,labels = tensorGenReg(deg=2)
features = features[:, :-1] # 剔除最后全是1的列
data = TensorDataset(features,labels) # 数据封装
batchData = DataLoader(data,batch_size = batch_size,shuffle = True) # 数据加载

# 定义模型

class LR(nn.Module):
    def __init__(self,in_features=2,out_features=1): # 定义模型的点线结构
        super(LR,self).__init__()
        self.linear = nn.Linear(in_features,out_features)
        
    def forward(self,x):
        out = self.linear(x)
        
        return out

# 实例化模型
LR_model = LR()

# 定义损失函数

criterion = nn.MSELoss()

# 定义优化方法

optimizer = optim.SGD(LR_model.parameters(), lr=0.03)

# 模型训练

def fit(net,criterion,optimizer,batchdata,epochs):
    for epoch in range(epochs):
        for X,y in batchdata:
            yhat = net.forward(X)
            loss = criterion(yhat,y)
            
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
        writer.add_scalar('loss',loss,global_step=epoch)
        
# 执行模型训练

writer = SummaryWriter(log_dir='reg_loss')

# 设置随机数种子
torch.manual_seed(420)

fit(net=LR_model,
   criterion = criterion,
   optimizer = optimizer,
   batchdata = batchData,
   epochs = num_epochs)

# 计算MSE
criterion(LR_model(features),labels)

tensor(10.0552, grad_fn=<MseLossBackward>)

In [89]:
# delta调大


# 定义核心参数
batch_size = 10 # 每一个小批量数
lr = 0.03 # 学习率
num_epochs = 3 # 训练遍历次数

# 数据准备
torch.manual_seed(420)

# 创建数据集
features,labels = tensorGenReg(delta=2)
features = features[:, :-1] # 剔除最后全是1的列
data = TensorDataset(features,labels) # 数据封装
batchData = DataLoader(data,batch_size = batch_size,shuffle = True) # 数据加载

# 定义模型

class LR(nn.Module):
    def __init__(self,in_features=2,out_features=1): # 定义模型的点线结构
        super(LR,self).__init__()
        self.linear = nn.Linear(in_features,out_features)
        
    def forward(self,x):
        out = self.linear(x)
        
        return out

# 实例化模型
LR_model = LR()

# 定义损失函数

criterion = nn.MSELoss()

# 定义优化方法

optimizer = optim.SGD(LR_model.parameters(), lr=0.03)

# 模型训练

def fit(net,criterion,optimizer,batchdata,epochs):
    for epoch in range(epochs):
        for X,y in batchdata:
            yhat = net.forward(X)
            loss = criterion(yhat,y)
            
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
        writer.add_scalar('loss',loss,global_step=epoch)
        
# 执行模型训练

writer = SummaryWriter(log_dir='reg_loss')

# 设置随机数种子
torch.manual_seed(420)

fit(net=LR_model,
   criterion = criterion,
   optimizer = optimizer,
   batchdata = batchData,
   epochs = num_epochs)

# 计算MSE
criterion(LR_model(features),labels)

tensor(4.0959, grad_fn=<MseLossBackward>)